## **Ant Colony Simulation**

### **Components of Simulation:**
- 1. Ant
- 2. Colony
- 3. Food
- 4. Main window
- 5. Nest
- 6. Parameters
- 7. Pheromones
- 8. Vectors

In [5]:
import pygame
from parameters import *
from vector import Vector
from math import pi, degrees, radians
import random

## **1. Ant**

In [7]:
class Ant:
    def __init__(self, position = Vector(), nest = None):
        self.position = position
        self.velocity = Vector()
        self.max_speed = 5
        self.trigger_radius = 12
        self.smell_radius = 35
        self.scavenger = Scavenger()
        self.nest = nest
        # Angling in Radiant
        self.angle = -self.velocity.Heading()
        self.color = white
        self.has_food = False
        self.isFollowingTrail = False
        
    def ReturnToNest(self, pheromone):
        if Vector.WithiinRange(self.position, self.nest.position, self.nest.radius):
            self.has_food = False
            self.nest.stock += 1
            self.color = white
            # self.angle = self.velocity.Heading()
            return
        self.velocity += self.scavenger.Seek(self.position, self.nest.position, self.velocity, self.max_speed)
        # Adding Randomness to mimmic more realistic movement
        wander_force = self.scavenger.Wander(self.velocity)
        self.velocity += (wander_force * 0.6)
        pher_direction = self.velocity.Negate()
        
        pheromone.AppendPheromone(self.position, pher_direction, "food")
        
    def SearchForFood(self, closest_food, pheromone):
        dist = Vector.GetDistance(self.position, closest_food.position)
        
        if dist < self.trigger_radius:
            self.TakeFood(closest_food)
        elif dist < self.smell_radius:
            self.Step(closest_food, pheromone)
        else:
            self.FollowPheromoneOrWander(pheromone)
        
        pheromone.AppendPheromone(self.position, self.velocity, "home")
        
    def UpdateVelocity(self, closest_food, pheromone):
        if self.has_food == True:
            self.ReturnToNest(pheromone)
        else:
            self.SearchForFood(closest_food, pheromone)
            
    def TakeFood(self, closes_food):
        self.has_food = True
        self.isFollowingTrail = False
        self.color = (220, 130, 30)
        closest_food.Bite()
        
    def Step(self, closest_food, pheromone):
        self.velocity += self.scavenger.Seek(self.position, closest_food.position, self.velocity, self.max_speed)
    
    def FollowPheromoneOrWander(self, pheromone):
        pheromone_direction = pheromone.PheromoneDirection(self.position, self.smell_radius, "food")
        pheromone_direction = pheromone_direction.Scale(self.max_speed)
        self.velocity = pheromone_direction
        self.velocity += self.scavenger.Wander(self.velocity)
        # pheromone.AppendPheromone(self.position, self.velocity, "home")
        
    def Update(self, foods, pheromones, dt):
        closest_food = foods.GetClosestFood(self.position)
        self.UpdateVelocity(cloeset_food, pheromones)
        self.velocity = self.velocity.Scale(self.max_speed)
        # self.position += self.velocity.Normalize() * dt * self.max_speed
        self.position += self.velocity
        self.angle = self.velocity.Heading()
        
    def Show(self, screen):
        # init Triangle point
        # rotate based on angle
        triangle = [
            (self.position + Vector(ant_size//2, 0).Rotate(self.angle)).xy(),
            (self.position - Vector(ant_size//2, - ant_size/3).Rotate(self.angle)).xy(),
            (self.position - Vector(ant_size//2, + ant_size/3).Rotate(self.angle)).xy()
        ]
        if self.has_food:
            pygrame.draw.circle(screen, (220, 130, 30), (self.position + Vector(ant_size/1.5, 0).Rotate(self.angle)).xy(), 2)
            
        pygame.draw.polygon(screen, self.color, triangle)
        
class Scavenger:
    def __init__(self):
        self.wander_distance = 20
        self.wander_radius = 12
        self.wander_angle = 1
        self.wander_delta_angle = pi/4
        
    def Seek(self, position, target, velocity, max_speed):
        diff = target - position
        diff = diff.Scale(max_speed)
        return diff - velocity
    
    def Wander(self, velocity):
        pos = velocity.Copy()
        pos = pos.Scale(self.wander_distance)
        displacement = Vector(0, -1).Scale(self.wander_radius)
        displacement = displacement.SetAngle(self.wander_angle)
        self.wander_angle += random.uniform(0, 1) * self.wander_delta_angle - self.wander_delat_angle * 0.5
        return pos + displacement